In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 모델 로드
model = BertForSequenceClassification.from_pretrained('./saved_model')
tokenizer = BertTokenizer.from_pretrained('./saved_model')

# 모델 학습
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

# 데이터셋 정의
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        label = self.labels[idx]
        
        # 1~5 label을 True, 0 label을 False로 변환
        if label in [1, 2, 3, 4, 5]:
            label = 1
        else:
            label = 0
        
        return {
            'input_ids': inputs['input_ids'].flatten().to(device),
            'attention_mask': inputs['attention_mask'].flatten().to(device),
            'labels': torch.tensor(label, dtype=torch.long).to(device)
        }
    

# 데이터 로드
df = pd.read_csv('data/output.csv')

# 데이터를 1000개로 제한
X_train, X_test, y_train, y_test = train_test_split(df['text'].sample(1000), df['types'].sample(1000), test_size=0.2, random_state=42)

train_dataset = HateSpeechDataset(X_train.tolist(), y_train.tolist(), tokenizer)
test_dataset = HateSpeechDataset(X_test.tolist(), y_test.tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


# 평가
model.eval()
predictions, true_labels = [], []
num = 0
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    batch_predictions = torch.argmax(logits, dim=1).tolist()
    batch_true_labels = batch['labels'].tolist()
    
    for i, (text, pred, true_label) in enumerate(zip(X_test, batch_predictions, batch_true_labels)):
        if pred == true_label:
            num += 1
        print(f"Text: {text}")
        print(f"Predicted Label: {pred}")
        print(f"True Label: {true_label}")
        print()

accuracy = num / len(X_test)
print(f"Accuracy: {accuracy:.2f}")


Text: 곧 지진 큰거 한방 갈 거야.
Predicted Label: 1
True Label: 0

Text: 넌 연애를 못 할 와꾸라 갑,을 따질 것도 없겠다
Predicted Label: 0
True Label: 1

Text: 꼭 처맞을 짓 하는 사람들이 폭력 반대하더라~
Predicted Label: 0
True Label: 1

Text: 근데 왜 사람을 그지경으로 만드셨습니까
Predicted Label: 1
True Label: 1

Text: 술자리에서 술 안 마시는 애들은 왜 나오는 거냐?
Predicted Label: 1
True Label: 0

Text: 밟지 말고 똑같이 하자 찢어 죽여야돼
Predicted Label: 0
True Label: 1

Text: 뭘 그리 돌려돌려 말하시나요
Predicted Label: 0
True Label: 1

Text: 뭐임?
Predicted Label: 1
True Label: 0

Text: 응원이라도 해라 쫌
Predicted Label: 1
True Label: 1

Text: 오래도록 친구였는데. 어쩌다 이별했니.
Predicted Label: 1
True Label: 0

Text: 이딴 걸 파는 짱깨들은 양심이 나가 뒤진 거냐?
Predicted Label: 0
True Label: 1

Text: 그래도 #@# 덕에 성평등 지수 많이 올랐을걸?
Predicted Label: 1
True Label: 0

Text: 야, 너한테나 귀여운 거지, 난 이제 걔 사진이 지겹고 짜증나거든.
Predicted Label: 0
True Label: 0

Text: 그냥 꼴보기 싫게 생겼잖아.
Predicted Label: 0
True Label: 0

Text: 트랜스 젠더는 정신병이야. ㅋㅋㅋ
Predicted Label: 0
True Label: 1

Text: 그런 소리하면 공안한테 잡혀갈지도 몰라
Predicted Label: 0
True Label: 1

Text: 곧 지진 큰거 한방